In [3]:
!pip install s3fs

  Created wheel for aiobotocore: filename=aiobotocore-2.1.2-py3-none-any.whl size=55983 sha256=e2421396b19c83a13b771851924da163cfba3d85666dd64f4c0ee61dfd6dca08
  Stored in directory: c:\users\taylo\appdata\local\pip\cache\wheels\a3\89\13\9c60f1de355c526583bb1c5712186fddd8f98f31ca1d78dc19
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3165 sha256=96696235da68762343fa80ad69d15929542208138166b6115414ddc1a86dee7c
  Stored in directory: c:\users\taylo\appdata\local\pip\cache\wheels\6a\f5\9c\f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
Successfully built aiobotocore idna-ssl


In [4]:
import json
from pathlib import Path
import os

import pandas as pd
import s3fs


def read_cluster_csv(file_path, endpoint_url='https://storage.budsc.midwest-datascience.com'):
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    return pd.read_csv(s3.open(file_path, mode='rb'))

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)

people_json = kv_data_dir.joinpath('people.json')
visited_json = kv_data_dir.joinpath('visited.json')
sites_json = kv_data_dir.joinpath('sites.json')
measurements_json = kv_data_dir.joinpath('measurements.json')

In [5]:
class KVDB(object):
    def __init__(self, db_path):
        self._db_path = Path(db_path)
        self._db = {}
        self._load_db()

    def _load_db(self):
        if self._db_path.exists():
            with open(self._db_path) as f:
                self._db = json.load(f)

    def get_value(self, key):
        return self._db.get(key)

    def set_value(self, key, value):
        self._db[key] = value

    def save(self):
        with open(self._db_path, 'w') as f:
            json.dump(self._db, f, indent=2)

In [12]:
def create_sites_kvdb():
    db = KVDB(sites_json)
    df = pd.read_csv('C:/Users/taylo/OneDrive/Documents/dsc650/data/external/tidynomicon/site.csv')
    for site_id, group_df in df.groupby('site_id'):
        db.set_value(site_id, group_df.to_dict(orient='records')[0])
    db.save()


def create_people_kvdb():
    db = KVDB(people_json)
    ## TODO: Implement code
    df = pd.read_csv('C:/Users/taylo/OneDrive/Documents/dsc650/data/external/tidynomicon/person.csv')
    for person_id, group_df in df.groupby('person_id'):
        db.set_value(person_id, group_df.to_dict(orient='records')[0])
    db.save()


def create_visits_kvdb():
    db = KVDB(visited_json)
    ## TODO: Implement code
    df = pd.read_csv('C:/Users/taylo/OneDrive/Documents/dsc650/data/external/tidynomicon/visited.csv')
    key = str(str(df['visit_id']) + str(df['site_id']))
    value = df['visit_date']
    for key, value in df.iterrows():
        db.set_value(key, value)
    db.save()


def create_measurements_kvdb():
    db = KVDB(measurements_json)
    ## TODO: Implement code
    df = pd.read_csv('C:/Users/taylo/OneDrive/Documents/dsc650/data/external/tidynomicon/measurements.csv')
    key = str(str(df['visit_id']) + str(df['person_id']) + str(df['quantity']))
    value = df['reading']
    for key, value in df.iterrows():
        db.set_value(key, value)
    db.save()

In [10]:
visits = pd.read_csv('C:/Users/taylo/OneDrive/Documents/dsc650/data/external/tidynomicon/visited.csv')
visits.head()

,visit_id,site_id,visit_date
0,619,DR-1,1927-02-08
1,622,DR-1,1927-02-10
2,734,DR-3,1930-01-07
3,735,DR-3,1930-01-12
4,751,DR-3,1930-02-26


In [11]:
m = pd.read_csv('C:/Users/taylo/OneDrive/Documents/dsc650/data/external/tidynomicon/measurements.csv')
m.head()

,visit_id,person_id,quantity,reading
0,619,dyer,rad,9.82
1,619,dyer,sal,0.13
2,622,dyer,rad,7.80
3,622,dyer,sal,0.09
4,734,pb,rad,8.41


In [13]:
create_sites_kvdb()
create_people_kvdb()
create_visits_kvdb()
create_measurements_kvdb()

TypeError: Object of type 'Series' is not JSON serializable